<h1><center>CommonLit EDA + Literature</center></h1>
                                                      
<center><img src = "https://images.unsplash.com/photo-1507842217343-583bb7270b66?ixid=MnwxMjA3fDB8MHxzZWFyY2h8MXx8bGlicmFyeXxlbnwwfHwwfHw%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=500&q=60" width = "750" height = "500"/></center>


<h4><center>🔥🔥☠️ If you find this notebook useful and resourceful, do leave behind a upvote. I will be updating this notebook on a regular basis, so please check back once new version comes up. ☠️🔥🔥</center>

## Error Evaluation Martix

<center><img src = "https://www.gstatic.com/education/formulas2/355397047/en/root_mean_square_deviation.svg" width = "550" height = "150"/></center>


**RMSD** &emsp; =&ensp;root-mean-square deviation/error<br>
**i** &emsp; &emsp; &emsp; =&ensp;variable i<br>
**N** &emsp; &emsp; &emsp;=&ensp;number of non-missing data points<br>
**x**<sub>i</sub> &emsp; &emsp; &emsp;=&ensp;actual observations time series<br>
**$\hat{x}$<sub>i</sub>** &emsp; &emsp; &ensp; =&ensp;estimated time series<br>

> The root-mean-square deviation (RMSD) or root-mean-square error (RMSE) is a frequently used measure of the differences between values (sample or population values) predicted by a model or an estimator and the values observed. The RMSD represents the square root of the second sample moment of the differences between predicted values and observed values or the quadratic mean of these differences. These deviations are called residuals when the calculations are performed over the data sample that was used for estimation and are called errors (or prediction errors) when computed out-of-sample. The RMSD serves to aggregate the magnitudes of the errors in predictions for various data points into a single measure of predictive power. RMSD is a measure of accuracy, to compare forecasting errors of different models for a particular dataset and not between datasets, as it is scale-dependent.

> RMSD is always non-negative, and a value of 0 (almost never achieved in practice) would indicate a perfect fit to the data. In general, a lower RMSD is better than a higher one. However, comparisons across different types of data would be invalid because the measure is dependent on the scale of the numbers used.

> RMSD is the square root of the average of squared errors. The effect of each error on RMSD is proportional to the size of the squared error; thus larger errors have a disproportionately large effect on RMSD. Consequently, RMSD is sensitive to outliers.

In [ ]:
from IPython.core.display import display, HTML, Javascript

def ApplyCustomCSS():
    return HTML("<style>"+open("../input/customcss/custom_kaggle_forCommit.css", "r").read()+"</style>")

ApplyCustomCSS()

In [ ]:
! pip install pyspellchecker 
! pip install textstat
import numpy as np
import pandas as pd
import warnings as w
from scipy import spatial
from tqdm.notebook import tqdm
from textblob import TextBlob
import random, math, cv2, os, string, re, gc, nltk, textstat
from collections import Counter
from spellchecker import SpellChecker

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline
sns.set(style="whitegrid")
w.filterwarnings('ignore')

SEED = 100

sample_sub = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
train.__name__ = "Train"
test.__name__ = "Test"

# nltk.download('stopwords')

train.head()

## Data Information with EDA

#### 📓 Data:
* train.csv - the training set<br>
* test.csv - the testing set

#### **Columns** :-
* **id** - unique ID for excerpt
* **url_legal** - URL of source
* **license** - license of source material
* **excerpt** - text to predict reading ease of
* **target** - reading ease
* **standard_error** - measure of spread of scores among multiple raters for each excerpt


#### Target 🎯 : Rating the complexity of the reading passages for 3-12 standards.

In [ ]:
length = train.excerpt.apply(lambda x: len(x))
print('\033[36m'+f"The training set size is {train.shape[0]} with {train.shape[1]} columns"+'\033[0m')
print('\033[32m'+f"The training set size is {test.shape[0]} with {test.shape[1]} columns"+'\033[0m')
print('\033[33m'+f"The training set maximum excerpt length is {max(length)} and with minimum length of {min(length)}"+'\033[0m')
train.target.describe()

In [ ]:
for df in [train, test]:
    print('\033[36m'+df.__name__+'\033[0m')
    for col in df.columns:
        print('\033[32m'+f"The {col} in {df.__name__} has "+'\033[0m'+'\033[33m'+f"{df[col].isnull().sum()} null values"+'\033[0m'+'\033[32m'+f" against {df.shape[0]}"+'\033[0m')
    print("")

In [ ]:
def plot_table(df, title_text, footer_text = 'May 7, 2021', fig_background_color = 'skyblue', fig_border = 'steelblue'):
    column_headers = df.columns
    row_headers = df.index

    cell_text = df.values

    rcolors = plt.cm.BuPu(np.full(len(row_headers), 0.1))
    ccolors = plt.cm.BuPu(np.full(len(column_headers), 0.1))

    plt.figure(linewidth=2,
               edgecolor=fig_border,
               facecolor=fig_background_color,
               tight_layout={'pad':1},

              )

    the_table = plt.table(cellText=cell_text,
                          rowLabels=row_headers,
                          rowColours=rcolors,
                          rowLoc='right',
                          colColours=ccolors,
                          colLabels=column_headers,
                          loc='center')

    the_table.scale(1, 1.5)

    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    plt.box(on=None)

    plt.suptitle(title_text)

    plt.figtext(0.95, 0.05, footer_text, horizontalalignment='right', size=6, weight='light')
    plt.draw()

def add_new_columns(text_row):
    text = text_row.excerpt
    processed_text = text_row.processed_excerpt
    
    punctuations_ = [word for word in text if word in string.punctuation]
    stopwords_ = [word for word in text.split() if word in set(nltk.corpus.stopwords.words("english"))]
    tokens = nltk.word_tokenize(text)
    sent_ = nltk.sent_tokenize(text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               "]+", flags=re.UNICODE)
    emojis_ = re.findall(emoji_pattern, text)
    misspelled_words = SpellChecker().unknown(processed_text.split())
    
    return pd.Series({"num_of_punctuations" : len(punctuations_), 
                      "num_of_stopwords" : len(stopwords_), 
                      "num_of_words" : len(tokens),
#                       "num_of_emojis" : len(emojis_),
                      "num_of_sentences" : len(sent_),
                      "num_of_misspelled_words" : len(misspelled_words),
                      "num_of_characters" : len(processed_text),
                      "num_of_unique_words" : len(set(tokens))})  

def clean(text):
    text = str(text).lower()
    text = ''.join([k for k in text if k not in string.punctuation]) # try by not removing punctuation as they are an important part of an essay 
    text = ' '.join([word for word in text.split() if word not in set(nltk.corpus.stopwords.words("english"))])
    
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(' +', ' ', text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text  

def n_gram_freq(text, n_grams, min_freq):
    result = {}
    tokens = nltk.word_tokenize(text)
    
    if n_grams == 2:
        ngram_toks = nltk.bigrams(tokens)
        
    elif n_grams == 3:
        ngram_toks = nltk.trigrams(tokens)
        
    else:
        ngram_toks = text.split()
        
    for key, val in Counter(ngram_toks).items():
        if n_grams == 1:
            result[key] = val
            continue
        if val > min_freq:
            result[' '.join(key)] = val
    
    return result

def plot_wordcloud(text_df, title, more_than_unigram = False, ):
    if more_than_unigram:
        text_df = {"_".join(words.split()):freq for words, freq in text_df.items()}
        wordcloud = WordCloud(background_color='white', stopwords=STOPWORDS, width=2560, height=1440).generate_from_frequencies(text_df)
        
    else:
        wordcloud = WordCloud(background_color='white', stopwords=STOPWORDS, width=2560, height=1440).generate(text_df)

    ax = plt.subplots(figsize=(15, 15), facecolor='w')
    plt.title(title, font="Serif", fontsize = 30, color = 'blue')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()
    
tqdm.pandas()
train['processed_excerpt'] = train['excerpt'].progress_apply(clean)
test["processed_excerpt"] = test['excerpt'].progress_apply(clean)
text = " ".join([record for record in train['excerpt']])
processed_text = " ".join([record for record in train['processed_excerpt']])

train[["num_of_punctuations",
       "num_of_stopwords",
       "num_of_words",
       "num_of_sentences",
       "num_of_misspelled_words",
       "num_of_characters",
       "num_of_unique_words"]] = train.progress_apply(add_new_columns, axis = 1)
test[["num_of_punctuations",
       "num_of_stopwords",
       "num_of_words",
       "num_of_sentences",
       "num_of_misspelled_words",
       "num_of_characters",
       "num_of_unique_words"]] = test.progress_apply(add_new_columns, axis = 1)

In [ ]:
sns.set_style("whitegrid", {'axes.grid' : False})
sns.pairplot(train[["num_of_punctuations",
                    "num_of_stopwords",
                    "num_of_words",
                    "num_of_sentences",
                    "num_of_misspelled_words",
                    "num_of_characters", 
                    "target"]])
#             corner=True)
#             plot_kws=dict(marker="*", linewidth=1))
#             diag_kws=dict(fill=False))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,6))
sns.distplot(train['target'], color = "#3F88C5",ax=ax[0], kde = True, kde_kws=dict(color = "#136F63", linewidth=3))
sns.distplot(train['standard_error'], color = "#136F63", kde = True,ax=ax[1], kde_kws=dict(color = "#3F88C5", linewidth=3))
ax[0].set_title("Target Distribution",font="Serif")
ax[1].set_title("Standard Error Distribution",font="Serif")
plt.show()

In [ ]:
# Unigram wordcloud and 
unigram_freq = n_gram_freq(processed_text, 1, 2)
uni_df = pd.DataFrame(data = unigram_freq.items(), columns = ["Unigram_word","Count"]).sort_values(by = "Count", ascending = False)
uni_df.reset_index(drop = True, inplace = True)
uni_df["Count"] = uni_df["Count"].astype(str)

plot_table(uni_df.head(10), "Top 10 most Occuring words")
plot_wordcloud(processed_text, "UniGram WordCloud", more_than_unigram = False)

In [ ]:
# Bigram wordcloud 
bigram_freq = n_gram_freq(processed_text, 2, 2)
bi_df = pd.DataFrame(data = bigram_freq.items(), columns = ["Bigram_word","Count"]).sort_values(by = "Count", ascending = False)
bi_df.reset_index(drop = True, inplace = True)
bi_df["Count"] = bi_df["Count"].astype(str)

plot_table(bi_df.head(10), "Top 10 most Occuring BiGrams")
plot_wordcloud(n_gram_freq(processed_text, 2, 2), "BiGram WordCloud", more_than_unigram = True)

In [ ]:
# Trigram wordcloud 
trigram_freq = n_gram_freq(processed_text, 3, 1)
tri_df = pd.DataFrame(data = trigram_freq.items(), columns = ["Trigram_word","Count"]).sort_values(by = "Count", ascending = False)
tri_df.reset_index(drop = True, inplace = True)
tri_df["Count"] = tri_df["Count"].astype(str)

plot_table(tri_df.head(10), "Top 10 most Occuring TriGrams")
plot_wordcloud(n_gram_freq(processed_text, 3, 1), "TriGram WordCloud", more_than_unigram = True)

In [ ]:
train['polarity']=train['processed_excerpt'].apply(lambda x:TextBlob(x).sentiment.polarity)
test['polarity']=test['processed_excerpt'].apply(lambda x:TextBlob(x).sentiment.polarity)

print("3 Random excerpts with Highest Polarity:")
for index,excerpt in enumerate(train.iloc[train['polarity'].sort_values(ascending=False)[:3].index]['processed_excerpt']):
    print('Excerpt {}:\n'.format(index+1),excerpt)

In [ ]:
train['dale_chall_score']=train['processed_excerpt'].apply(lambda x: textstat.dale_chall_readability_score(x))
train['flesh_reading_ease']=train['processed_excerpt'].apply(lambda x: textstat.flesch_reading_ease(x))
train['gunning_fog']=train['processed_excerpt'].apply(lambda x: textstat.gunning_fog(x))

test['dale_chall_score']=test['processed_excerpt'].apply(lambda x: textstat.dale_chall_readability_score(x))
test['flesh_reading_ease']=test['processed_excerpt'].apply(lambda x: textstat.flesch_reading_ease(x))
test['gunning_fog']=test['processed_excerpt'].apply(lambda x: textstat.gunning_fog(x))

print('Dale Chall Score ',train['dale_chall_score'].mean())

print('Flesch Reading Score ',train['flesh_reading_ease'].mean())

print('Gunning Fog Index ',train['gunning_fog'].mean())


In [ ]:
def find_best_model(models_list, x_train, y_train, x_test, y_test, use_text = True):
    if use_text:
        print("\t Predicting with processed text")
    else:
        print("\t Predicting with information from processed text")
    for mod in models_list:
        if use_text:
            model = make_pipeline(
                TfidfVectorizer(binary=True, ngram_range=(1,1)),
                mod)
        else:
            model = mod
            
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        MSE = mean_squared_error(y_test, y_pred)
        
        print("\t Model Name:", mod.__name__)
        print("MSE: ", MSE, end = "\n")
    print("--*"*10 )

ridge = Ridge(fit_intercept = True, normalize = False)
lr = LinearRegression()
mlpreg = MLPRegressor(hidden_layer_sizes=(3,), activation='relu',
         solver='adam', alpha=0.001, batch_size='auto',
         learning_rate='adaptive', learning_rate_init=0.01,
         power_t=0.5, max_iter=1000, shuffle=True, random_state=9,
         tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
         nesterovs_momentum=True, early_stopping=False,
         validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
         epsilon=1e-08)


ridge.__name__ = "Ridge Regression"
lr.__name__ = "Linear Regression"
mlpreg.__name__ = "MLPRegressor"
models = [ridge,lr,mlpreg]

X1 = train["processed_excerpt"]
y1 = train['target']
X2 = train[["num_of_punctuations", "num_of_stopwords", "num_of_words", "dale_chall_score", "flesh_reading_ease", "gunning_fog",
               "num_of_unique_words", "num_of_sentences", "num_of_misspelled_words", "num_of_characters", "polarity"]]
y2 = train['target']

x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.33, random_state=2)
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.33, random_state=2)

find_best_model(models, x_train1, y_train1, x_test1, y_test1, use_text = True)
find_best_model(models, x_train2, y_train2, x_test2, y_test2, use_text = False)

In [ ]:
def get_final_preds(mod,X,y, use_text, features = None):
    if use_text:
        model = make_pipeline(
            TfidfVectorizer(binary=True, ngram_range=(1,1)),
            mod)
        model.fit(X, y)
        preds = model.predict(test["processed_excerpt"])
    else:
        mod.fit(X[features], y)
        preds = mod.predict(test[features])

    return preds

test_pred = get_final_preds(mlpreg, train, train['target'], False, features = ["num_of_punctuations", "num_of_stopwords", "num_of_words", "dale_chall_score", "flesh_reading_ease", "gunning_fog",
               "num_of_unique_words", "num_of_sentences", "num_of_misspelled_words", "num_of_characters", "polarity"])
submission = pd.DataFrame()
submission['id'] = test['id']
submission['target'] = test_pred
submission.to_csv("submission.csv", index=False)
submission